In [1]:
# compute channel frequency
# compute_ch_freq_v2.ipynb

# 2021.02.19
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:93% !important; }</style>"))

In [2]:
import math
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import numpy as np
import os, sys
import time
import pickle 
import gc
import re
gc.collect()

0

# https://www.etsi.org/deliver/etsi_ts/136500_136599/13652101/15.05.00_60/ts_13652101v150500p.pdf
# 5.4.4 Carrier frequency and EARFCN 
# Table 5.4.4-1: E-UTRA channel numbers

$\displaystyle F_{DL} = F_{DL_{low}} + 0.1 ( N_{DL} – N_{Offs-DL} ) $

In [3]:
def freq_dl(  Fdl_low, Ndl, Noffs_dl  ):
    return Fdl_low + 0.1*( Ndl - Noffs_dl )
    
fn = 'TS_13652101v150500p_Tbl5.4.4_1E_UTRA_Ch_nums.xlsx'
d = pd.read_excel(fn, sheet_name='ch_num_tbl')
print(d.shape)

# drop rows where band is not present
ix = [ type(d['band'][i])==int  for i in range(len(d)) ]
d=d.loc[ix].reset_index(drop=True)
print(d.shape)

# https://stackoverflow.com/questions/17336943/removing-non-numeric-characters-from-a-string
import re
# d['Ndl_min'] = [ re.sub('[^0-9]',' ', d['Ndl'][i]).strip().split(' ')[0]    for i in range(len(d)) ]
# d['Ndl_max'] = [ re.sub('[^0-9]',' ', d['Ndl'][i]).strip().split(' ')[-1] for i in range(len(d)) ]

d['Ndl_min'] = [ int(re.sub('[^0-9]',' ', d['Ndl'][i]).strip().split(' ')[ 0])    for i in range(len(d)) ]
d['Ndl_max'] = [ int(re.sub('[^0-9]',' ', d['Ndl'][i]).strip().split(' ')[-1])    for i in range(len(d)) ]

# d.head()

(65, 10)
(62, 10)


In [4]:
# !dir

In [5]:
d.head()

,band,Fdl_low_MHz,Noff_dl,Ndl,Ful_low_MHz,Noff_ul,Nul,band_w_idx,com_num,comment,Ndl_min,Ndl_max
0,1,2110.0,0.0,0 – 599,1920.0,18000.0,18000 – 18599,1,NaN,NaN,0,599
1,2,1930.0,600.0,600 - 1199,1850.0,18600.0,18600 – 19199,2,NaN,NaN,600,1199
2,3,1805.0,1200.0,1200 – 1949,1710.0,19200.0,19200 – 19949,3,NaN,NaN,1200,1949
3,4,2110.0,1950.0,1950 – 2399,1710.0,19950.0,19950 – 20399,4,NaN,NaN,1950,2399
4,5,869.0,2400.0,2400 – 2649,824.0,20400.0,20400 – 20649,5,NaN,NaN,2400,2649


In [6]:
# EXAMPLE
Ndl =  2600
ix = (d['Ndl_min'] <= Ndl) & (Ndl<=d['Ndl_max'])

[band, Fdl_low_MHz, Noff_dl] = np.array(d.loc[ix][ ['band', 'Fdl_low_MHz', 'Noff_dl'] ])[0]


freq_MHz = freq_dl(  Fdl_low=Fdl_low_MHz, Ndl=Ndl, Noffs_dl=Noff_dl  )

print(f' earfcn_dl={Ndl},  band={band},   Freq_MHz={freq_MHz}')

# ix
np.sum(ix)
d.loc[ix]

 earfcn_dl=2600,  band=5,   Freq_MHz=889.0


,band,Fdl_low_MHz,Noff_dl,Ndl,Ful_low_MHz,Noff_ul,Nul,band_w_idx,com_num,comment,Ndl_min,Ndl_max
4,5,869.0,2400.0,2400 – 2649,824.0,20400.0,20400 – 20649,5,NaN,NaN,2400,2649


In [7]:
# d.columns

# def freq_dl(  Fdl_low, Ndl, Noffs_dl  ):
#     return Fdl_low + 0.1*( Ndl - Noffs_dl )
def get_band_freqDL( d, Ndl ):
    ix = (d['Ndl_min'] <= Ndl) & (Ndl<=d['Ndl_max'])
    if np.sum(ix) == 1:
        [band, Fdl_low_MHz, Noff_dl] = np.array(d.loc[ix][ ['band', 'Fdl_low_MHz', 'Noff_dl'] ])[0]
        freq_MHz = Fdl_low_MHz + 0.1*( Ndl - Noff_dl )
    else:
        band, freq_MHz = -1,-1   #'NA', 'NA'
        
    return band, freq_MHz


Ndl =  2600
band, freq_MHz = get_band_freqDL( d, Ndl )
print(f' earfcn_dl={Ndl},  band={band},   Freq_MHz={freq_MHz}')

Ndl = earfcn_dl = 50090
band, freq_MHz = get_band_freqDL( d, Ndl )
print(f' earfcn_dl={Ndl},  band={band},   Freq_MHz={freq_MHz}')

Ndl = earfcn_dl = 175000
band, freq_MHz = get_band_freqDL( d, Ndl )
print(f' earfcn_dl={Ndl},  band={band},   Freq_MHz={freq_MHz}')
    
# freq_dl(  Fdl_low=Fdl_low_MHz, Ndl=Ndl, Noffs_dl=Noff_dl  )

 earfcn_dl=2600,  band=5,   Freq_MHz=889.0
 earfcn_dl=50090,  band=46,   Freq_MHz=5480.0
 earfcn_dl=175000,  band=-1,   Freq_MHz=-1


# Calculate BAND and Freq_MHz for all Ndl values 

In [8]:
# del d2

In [9]:
d2 = pd.read_excel(fn, sheet_name='Ndl')
d2['band'] = 0
d2['freq_dl_MHz'] = 0
print(d2.shape)
d2.head()

(9, 3)


,Ndl,band,freq_dl_MHz
0,0,0,0
1,1025,0,0
2,2375,0,0
3,2600,0,0
4,5230,0,0


In [10]:
for i in range(len(d2)):
    Ndl = d2.Ndl[i]
#     print(f'======> i={i}   Ndl={Ndl}')
    band, freq_MHz = get_band_freqDL( d, Ndl )
    d2.band[i] = band
    d2.freq_dl_MHz[i] = freq_MHz    

In [11]:
print(d2.shape)
d2

(9, 3)


,Ndl,band,freq_dl_MHz
0,0,1,2110
1,1025,2,1972
2,2375,4,2152
3,2600,5,889
4,5230,13,751
5,53541,46,5825
6,55040,47,5905
7,56690,48,3695
8,66461,66,2112


In [14]:
# i=5
# Ndl=d2.Ndl[i]
# band, freq_MHz = get_band_freqDL( d, Ndl )
# print(f' Ndl={Ndl},  band={band},   Freq_MHz={freq_MHz}')


# ix = (d['Ndl_min'] <= Ndl) & (Ndl<=d['Ndl_max'])
# ix
# assert np.sum(ix) == 1

# # [band, Fdl_low_MHz, Noff_dl] =
# np.array(d.loc[ix][ ['band', 'Fdl_low_MHz', 'Noff_dl'] ])[0]
# #     freq_MHz = Fdl_low_MHz + 0.1*( Ndl - Noff_dl )
# # else:
# #     band, freq_MHz = -1,-1   #'NA', 'NA'

In [ ]:
# d2.to_csv('Ndl_band_freqMHz.csv', index=False)